# Automate Market Research


This AI tool is

### Environment: Googloe Colab,

1.  啟動前，先在secret keys選項中變更`episode_folder_name`, `starting_date`, `ending_date`
2.  先啟動上半部分>>挑選好主題 >>啟動下半部分


In [ ]:
pip install newsapi-python

In [ ]:
import gspread
from google.oauth2.service_account import Credentials
from newsapi import NewsApiClient
import json

from bs4 import BeautifulSoup
import requests
from googleapiclient.discovery import build
from datetime import datetime

import time

from google.colab import userdata
import os

In [ ]:
# Define the scope and authorize the credentials
SCOPES = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
credentials_path = '/content/flying-chicken-433105-b571fb548d0f.json'
creds = Credentials.from_service_account_file(credentials_path, scopes=SCOPES)

In [ ]:
# Initialize gspread client
client = gspread.authorize(creds)

# Part 1: Collect News

## Connect NewsAPI

In [ ]:
# Init
newsapi = NewsApiClient(api_key = userdata.get('news_api'))

## Build Functions

### Export to google sheet

In [ ]:
def export_to_google_sheet(all_articles, sheet_id, sheet_name):
    # Open the Google Sheet and add a new worksheet or access an existing one
    sheet = client.open_by_key(sheet_id)
    try:
        worksheet = sheet.worksheet(sheet_name)
        worksheet.clear()
    except gspread.exceptions.WorksheetNotFound:
        worksheet = sheet.add_worksheet(title=sheet_name, rows="100", cols="20")

    # Prepare the header
    header = ["Title", "Author", "Source", "Published At", "Description", "URL", "Content"]
    worksheet.append_row(header)

    # Prepare the data rows for batch update
    rows = []
    for article in all_articles['articles']:
        title = article.get('title', 'No Title')
        author = article.get('author', 'No Author')
        source = article['source']['name']
        published_at = article.get('publishedAt')
        description = article.get('description', 'No Description')
        url = article.get('url', 'No URL')
        content = article.get('content', 'No Content')

        rows.append([title, author, source, published_at, description, url, content])

    # Perform batch update
    if rows:
        worksheet.append_rows(rows)

    print(f"Articles exported successfully to the sheet: https://docs.google.com/spreadsheets/d/{sheet_id}/edit#gid={worksheet.id}")


### Query News API

In [ ]:
# Function to query NewsAPI for a given keyword
def query_news_api(keyword, starting_date, ending_date, pages):
    all_articles = newsapi.get_everything(q=keyword,
                                          sources=None,
                                          domains=None,
                                          from_param= starting_date,
                                          to= ending_date,
                                          language='en',
                                          sort_by='relevancy',
                                          page= pages)
    return all_articles

### Process multiple keywords

In [ ]:
def process_keywords(keywords, starting_date, ending_date, pages, sheet_id):
    for keyword in keywords:
        print(f"Processing keyword: {keyword}")
        # Query the News API for the current keyword
        all_articles = query_news_api(keyword, starting_date, ending_date, pages)

        # Use the keyword directly as the sheet name
        sheet_name = keyword  # Assuming the tab in the Google Sheet has the exact same name as the keyword

        # Export the articles to the specific sheet/tab named after the keyword
        export_to_google_sheet(all_articles, sheet_id, sheet_name)
        print(f"Finished processing for keyword: {keyword}")

## Run Main Function

In [ ]:
if __name__ == "__main__":
    keywords = ["AI and marketing","AI and media", "AI and innovation", "AI and business", "AI and management"]
    starting_date = userdata.get('starting_date')
    ending_date = userdata.get('ending_date')
    pages = 5
    sheet_id = userdata.get('sheet_id')

    process_keywords(keywords, starting_date, ending_date, pages, sheet_id)

Processing keyword: AI and marketing
Articles exported successfully to the sheet: https://docs.google.com/spreadsheets/d/1pvhylBx6ixPXGNs_-66gedMt-V7a1ggETSe44ztCAug/edit#gid=1695063662
Finished processing for keyword: AI and marketing
Processing keyword: AI and media
Articles exported successfully to the sheet: https://docs.google.com/spreadsheets/d/1pvhylBx6ixPXGNs_-66gedMt-V7a1ggETSe44ztCAug/edit#gid=499554999
Finished processing for keyword: AI and media
Processing keyword: AI and innovation
Articles exported successfully to the sheet: https://docs.google.com/spreadsheets/d/1pvhylBx6ixPXGNs_-66gedMt-V7a1ggETSe44ztCAug/edit#gid=368577039
Finished processing for keyword: AI and innovation
Processing keyword: AI and business
Articles exported successfully to the sheet: https://docs.google.com/spreadsheets/d/1pvhylBx6ixPXGNs_-66gedMt-V7a1ggETSe44ztCAug/edit#gid=978405254
Finished processing for keyword: AI and business
Processing keyword: AI and management
Articles exported successfull

# Part 2: Scrape Content

## Extract and open the filtered URLs

In [ ]:
# Extract URLs from a Google Sheet
def extract_urls_from_sheet(sheet_id, sheet_name):
    sheet = client.open_by_key(sheet_id)

    # Print all available sheet names for debugging
    worksheets = sheet.worksheets()
    available_sheet_names = [ws.title for ws in worksheets]
    print(f"Available sheet names: {available_sheet_names}")

    # Strip whitespace from the sheet name before accessing
    sheet_name = sheet_name.strip()

    if sheet_name not in available_sheet_names:
        raise ValueError(f"Sheet name '{sheet_name}' not found. Available sheet names: {available_sheet_names}")

    worksheet = sheet.worksheet(sheet_name)

    # The URLs are in a specific column, e.g., column F (index 6)
    urls = worksheet.col_values(6)[1:]  # Skip the header
    return urls

## Scrape web content

In [ ]:
# Scrape content from a URL and include the URL at the top
def scrape_content_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        response.encoding = 'utf-8'  # Ensure proper encoding
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract main content
        paragraphs = soup.find_all('p')
        if not paragraphs:
            # Fallback if no <p> tags found
            paragraphs = soup.find_all('div')
        article_content = ' '.join([p.get_text() for p in paragraphs])

        # Check if content is minimal
        if len(article_content) < 50:
            print(f"Warning: Content for {url} is very short or empty.")
            return None

        # Prepend the URL to the article content
        formatted_content = f"Original URL: {url}\n\n{article_content}"

        return formatted_content

    except requests.RequestException as e:
        print(f"Failed to retrieve content from {url}: {str(e)}")
        return None


In [ ]:
# Create a folder in Google Drive under a specified parent folder
def create_folder_in_drive(drive_service, folder_name, parent_folder_id):
    # Check if the folder already exists
    query = f"'{parent_folder_id}' in parents and mimeType='application/vnd.google-apps.folder' and name='{folder_name}'"
    results = drive_service.files().list(q=query).execute()
    folders = results.get('files', [])

    if folders:
        # Folder already exists, return its ID
        return folders[0]['id']
    else:
        # Create a new folder
        file_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder',
            'parents': [parent_folder_id]
        }
        folder = drive_service.files().create(body=file_metadata, fields='id').execute()
        return folder['id']


In [ ]:
# Create a Google Doc inside a specified folder
def create_google_doc_in_folder(content, title, folder_id):
    docs_service = build('docs', 'v1', credentials=creds)
    drive_service = build('drive', 'v3', credentials=creds)

    # Create a new Google Doc
    doc = docs_service.documents().create(body={"title": title}).execute()
    document_id = doc['documentId']

    # Add content to the Google Doc
    requests_body = [
        {
            'insertText': {
                'location': {
                    'index': 1,
                },
                'text': content
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests_body}).execute()

    # Move the document to the specified folder
    drive_service.files().update(fileId=document_id, addParents=folder_id, fields='id, parents').execute()

    return document_id

In [ ]:
# Share a Google Doc with a specific email address
def share_google_doc(document_id, email):
    drive_service = build('drive', 'v3', credentials=creds)

    user_permission = {
        'type': 'user',
        'role': 'writer',
        'emailAddress': email
    }
    drive_service.permissions().create(
        fileId=document_id,
        body=user_permission,
        fields='id',
        sendNotificationEmail=True  # This will send a notification email to the user
    ).execute()


In [ ]:
# Main function to process URLs and export each to an individual Google Doc
def process_urls_and_export_to_docs(sheet_id, sheet_names, user_email, parent_folder_id):
    current_date = datetime.now().strftime("%Y%m%d")

    # Authenticate and build the Google Drive API service
    drive_service = build('drive', 'v3', credentials=creds)

    # Create the "Episode 8" folder under the given parent folder
    episode_folder_name = userdata.get('episode_folder_name')
    folder_id = create_folder_in_drive(drive_service, episode_folder_name, parent_folder_id)
    print(f"Folder '{episode_folder_name}' created with ID: {folder_id}")

    # Create the "Original" folder under the "Episode 8" folder
    original_folder_name = "Original"
    original_folder_id = create_folder_in_drive(drive_service, original_folder_name, folder_id)
    print(f"Folder '{original_folder_name}' created with ID: {original_folder_id}")

    # Ensure sheet_names is a list
    if isinstance(sheet_names, str):
        sheet_names = [sheet_names]  # Convert string to list with one item

    # Loop through the sheet names to extract URLs and create docs
    for sheet_name in sheet_names:
        print(f"Processing sheet: {sheet_name}")

        # Extract URLs from the current sheet
        urls = extract_urls_from_sheet(sheet_id, sheet_name)

        for i, url in enumerate(urls, start=1):
            print(f"Processing URL {i}: {url}")
            content = scrape_content_from_url(url)
            if content:
                # Create Google Doc with "Original" prefix
                doc_title = f"Original- {i} - {current_date}"
                document_id = create_google_doc_in_folder(content, title=doc_title, folder_id=original_folder_id)
                share_google_doc(document_id, user_email)
                print(f"Document {doc_title} created and shared: https://docs.google.com/document/d/{document_id}/edit")
                time.sleep(1)  # Delay to avoid overwhelming the server
            else:
                print(f"Skipped creating document for URL {url}")

In [ ]:
if __name__ == "__main__":
    # Example usage with real data (replace with your actual data)
    sheet_id = userdata.get('sheet_id')  # Replace with your actual Google Sheet ID
    sheet_names = userdata.get('sheet_names')  # Replace with the actual sheet names you want to process
    user_email = userdata.get('email_address')  # Replace with the email you want to share the document with

    # Replace this with your desired parent folder ID (from your provided Google Drive folder link)
    parent_folder_id = '1ORlrqMM9wK3SUJUdIOB4yDq7p71_JL1z'  # This is the folder where "Episode 8" will be created

    # Call the main process to extract URLs, scrape content, and export each to an individual Google Doc
    process_urls_and_export_to_docs(sheet_id, sheet_names, user_email, parent_folder_id)

Folder 'Episode 20' created with ID: 1F-fqo6eMgkzK_EAKR7LsNSeTSd47Bvbv
Folder 'Original' created with ID: 1HaxLGZ7LoSlBEw1d9F0OBc08OdJNUQBt
Processing sheet: Top_picks
Available sheet names: ['Historical finalist', 'Top_picks', 'AI and marketing', 'AI and marketing scoring', 'AI and marketing filtered', 'AI and media', 'AI and media scoring', 'AI and media filtered', 'AI and innovation', 'AI and innovation scoring', 'AI and innovation filtered', 'AI and business', 'AI and business scoring', 'AI and business filtered', 'AI and management', 'AI and management scoring', 'AI and management filtered']
Processing URL 1: https://www.healthcareitnews.com/news/himsscast-how-genai-can-reinvent-work-clinicians
Document Original- 1 - 20250106 created and shared: https://docs.google.com/document/d/1mUsk3WHshPTrk13uh4o1C2snvfO_4FA3g1K3Bh-KFCw/edit
Processing URL 2: https://www.ibtimes.com/green-stays-digital-keys-virtual-tours-future-accommodation-now-3757565
Failed to retrieve content from https://

# Old Scripts

In [ ]:
# def create_google_doc(content, title):
#     docs_service = build('docs', 'v1', credentials=creds)

#     # Create a new Google Doc
#     doc = docs_service.documents().create(body={"title": title}).execute()
#     document_id = doc['documentId']

#     # Add content to the Google Doc
#     requests_body = [
#         {
#             'insertText': {
#                 'location': {
#                     'index': 1,
#                 },
#                 'text': content
#             }
#         }
#     ]

#     docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests_body}).execute()

#     return document_id